In [2]:
from global_methods import *
from global_constants import *

import pandas as pd
import matplotlib as plt
import threading
import seaborn as sb
from fastai.tabular.core import df_shrink
import datetime
import win32file
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat
import csv

c:\Users\Willi Wu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
win32file._setmaxstdio(6000)
def join_all_data(name, filepath, lock):
    output = open(name,'a', encoding="utf-8")
    raw_df = pd.read_csv(filepath, on_bad_lines="skip",encoding='utf-8')
    raw_df = raw_df.to_string(header=False,index=False)
    with lock:
        output.write(raw_df)
        raw_df = clean(raw_df)
        output.close() 

In [4]:
raw_csv_list = create_file_list(SPEND_PATTERNS, '.csv')
output_dir = "D:\\Code\\Safegraph_Project\\Testing\\Data\\initial_analysis.csv"

In [5]:
run = True
# runtime: 5 min
if run:
    lock = threading.Lock()
    threads = [threading.Thread(target=join_all_data,args=(output_dir, file, lock)) for file in raw_csv_list]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

In [ ]:
columns = [ 'placekey',
            'location_name',
            'naics_code',
            'street_address',
            'city',
            'region',
            'tracking_closed_since',
            'spend_date_range_start', 
            'spend_date_range_end',
            'raw_total_spend',
            'raw_num_transactions',
            'raw_num_customers']
names = [   'placekey',
            'parent_placekey'	
            'location_name'	
            'safegraph_brand_ids',	
            'brands',	
            'top_category',	
            'sub_category',	
            'naics_code',	
            'latitude',	
            'longitude',	
            'street_address',	
            'city',	
            'region',	
            'postal_code',	
            'iso_country_code',
            'phone_number',
            'open_hours',
            'category_tags',
            'opened_on',
            'closed_on',
            'tracking_closed_since',
            'geometry_type',
            'spend_date_range_start',
            'spend_date_range_end',
            'raw_total_spend',
            'raw_num_transactions',	
            'raw_num_customers',
            'median_spend_per_transaction',
            'median_spend_per_customer',
            'spend_per_transaction_percentiles',
            'spend_by_day',
            'spend_per_transaction_by_day',
            'spend_by_day_of_week',
            'day_counts',
            'spend_pct_change_vs_prev_month',
            'spend_pct_change_vs_prev_year',
            'online_transactions',
            'online_spend',	
            'transaction_intermediary',
            'spend_by_transaction_intermediary',
            'bucketed_customer_frequency',	
            'mean_spend_per_customer_by_frequency',	
            'bucketed_customer_incomes',	
            'mean_spend_per_customer_by_income',	
            'customer_home_city']

# Some Lines contain unique characters that cannot be encoded in utf-8. Like the copyright symbol
limited_view_df = pd.read_csv(output_dir, names=names, usecols=columns)

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max.columns',None)
pd.set_option('display.width',1000)
pd.set_option('display.colheader_justify','center')
pd.set_option('display.precision',3)

In [ ]:
# Q1: How unique are the placekeys? Are there multiple entries every month? Is there consistnecy across the years?
# 1. Spend_date_range_start and spend_date_range_end are going to be cleaned up. I only want YYYY-MM-DD
# 2. Create a new unique identifier -> merging placekey and the spend_date

# 3. If when I filter based off of this new id and I get a duplicate entry -> Indicates multiple entries every month
# 4. Expectation is that I see a single completely unique placekey for every month up until the business closes or temporarily closes

data_df = limited_view_df.copy()
# 762,721 unique placekeys out of 12 million records 
unique_placekeys = data_df['placekey'].unique()

time_split = ['spend_date_range_start', 'spend_date_range_end']
for col in time_split:
    data_df[col] = data_df[col].apply(lambda x: x.split('T')[0])
    data_df[col] = data_df[col].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d"))
    
data_df['new_placekey'] = str(data_df['placekey'][0]) + '-' + str(data_df['spend_date_range_start'])
first_column = data_df.pop('new_placekey')
data_df.insert(0, 'new_placekey', first_column)

In [ ]:
for placekey in unique_placekeys[0:1]:
    placekey_df = data_df[(data_df['placekey']==placekey)]
    display(placekey_df)

In [ ]:
limited_view_df['spend_date_range_start'].unique()

In [ ]:
data_df['spend_date_range_start'].unique()

In [ ]:
# weird = ['postal_code', 'tracking_closed_since', 'open_hours', '-149.869377', 'sub_category', ' ""18:00""]]']
# for index,row in data_df.iterrows():
#     value = row['spend_date_range_start']
#     if value in weird:
#         print(index, row['place_key'])

In [ ]:
from pre_processing import *
test_list = create_file_list(r'D:\Code\Safegraph_Project\SG_Data\spend_patterns\y=2021\m=3', '.csv')
total_test_df = pd.DataFrame()
for file in test_list:
    name_split = file_name_preprocessing(file)
    try:
        raw_df = pd.read_csv(file, on_bad_lines="skip")
    except:
        pass
    raw_df['source_location-year'] = name_split[0]
    raw_df['source_location-month'] = name_split[1]
    raw_df['source_location-part'] = name_split[2]
    total_test_df = pd.concat([total_test_df,raw_df])        